In [13]:
import os
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo #Planck 2015
import requests

In [14]:
TNG  = 50
lvl = 1
TNGpath = "/home/tnguser/sims.TNG/TNG%s-%s/output" % (TNG, lvl)
baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"e36226423a0cc5e62f2e553f39b44238"}
if TNG==50:
    rbox=35
elif TNG==100:
    rbox=75
elif TNG==300:
    rbox=205

In [15]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    return r

In [16]:
#get basic snapshots information (redshift, url) for each of the 100 snapshots

r = get(baseUrl) #path to all Illustris TNG simulations
names = [sim['name'] for sim in r['simulations']] #names of all available simulations
i = names.index('TNG%s-%s' % (TNG, lvl)) #index of specific simulation
sim = get( r['simulations'][i]['url'] ) #get path to that simulation
snaps = get( sim['snapshots'] ) #get path to snapshots for that simulation
nfiles = sim['num_files_snapshot'] #number files per snapshot
print(nfiles)

680


In [6]:
def get_z_and_lookbacktime(snaps):
    #get the redshifts for each snapshot
    redshifts = np.array([])
    for ind, val in enumerate(snaps):
        redshifts = np.append(redshifts, snaps[ind]['redshift'])
    
    #calculate lookback time for each snapshot
    lookbacktimes = np.array(cosmo.lookback_time(redshifts))
    
    return redshifts, lookbacktimes

In [7]:
#function to get binned SFR for each snapshot
def getSFRMetallicityFromGas(TNGpath, TNG, lvl, snap, nfiles, nBinsSFR=60):
    mbins = np.logspace(-10, 0., nBinsSFR+1)
    sfrs  = np.zeros((nBinsSFR))
    metals = np.zeros((nBinsSFR))
    
    fname =  TNGpath + "/snapdir_%03d/snap_%03d.%s.hdf5"%(snap, snap, "%d") 
    print(fname)
    
    for ifile in range(nfiles):
        with h5.File(fname % ifile, "r") as f:
            pStars = f["PartType0"]
      
            SFR = pStars["StarFormationRate"][:]
            Metals = pStars["GFM_Metallicity"][:]
      
        data, e = np.histogram(Metals, bins=mbins, weights=SFR)
        metaldata, e = np.histogram(Metals, bins=mbins)
        sfrs += data
        metals += metaldata
            
    return sfrs, metals

In [8]:
#function to make the data file; iterates through all snapshots

def getFullSFRMetallicityFromGas(TNGpath, TNG, lvl, snaps, nfiles, starting_snap, nbins=60):
    outfname = "TESTSFRMetallicityFromGasWithMetalsTNG%d-%d.hdf5" % (TNG,lvl)
    sfrs = np.zeros((len(snaps),nbins))
    metals = np.zeros((len(snaps),nbins))
    redshifts, lookbacktimes = get_z_and_lookbacktime(snaps)
        
    for snap in range(starting_snap, len(snaps)):
        if sfrs[snap].sum() == 0:
            print( "Doing snap %d." % snap )
            s, m = getSFRMetallicityFromGas(TNGpath, TNG, lvl, snap, nfiles, nBinsSFR=nbins)
            sfrs[snap,:] = s
            metals[snap,:] = m

        if snap==0: #if starting from beginning, assume no file exists to append into
            mbins = np.logspace( -10, 0., nbins+1 )
            if os.path.exists(outfname): #check if file with same name exists; if not append, does not overwrite it, makes a new separate file
                outfname = outfname + "-1"
            with h5.File(outfname, "w") as f:
                f.create_dataset('MetalBins', data=mbins )
                f.create_dataset('Redshifts', data=redshifts )
                f.create_dataset('Lookbacktimes', data=lookbacktimes )
                f.create_dataset('Sfr', data=sfrs)
                f.create_dataset('Metals', data=metals)

        else:
            f = h5.File(outfname, 'r+') # open existing file
            sfr_data = f['Sfr']       # load existing sfr data; assuming other cols already written
            sfr_data[snap] = sfrs[snap] #write new snapshot data to it; overwrites anything already in that column
            metal_data = f['Metals']       
            metal_data[snap] = metals[snap]
            f.close()

In [9]:
nfiles_temp = 6
starting_snap = 0
getFullSFRMetallicityFromGas(TNGpath, TNG, lvl, snaps, nfiles, starting_snap, nbins=60)

Doing snap 0.
/home/tnguser/sims.TNG/TNG50-1/output/snapdir_000/snap_000.%d.hdf5


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/tnguser/sims.TNG/TNG50-1/output/snapdir_000/snap_000.0.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
with h5.File("SFRMetallicityFromGasWithMetalsTNG%d-%d.hdf5" % (TNG,lvl), "r") as f:
        MetalBins     = f["MetalBins"][:]
        Lookbacktimes = f["Lookbacktimes"][:]
        BoxSfr        = f["Sfr"][:]
        Redshifts     = f["Redshifts"][:]
        Metals        = f["Metals"][:]

Sim_center_Zbin  = (MetalBins[:-1] + MetalBins[1:])/2.

# Convert SFR from sfr/box to sfr Mpc-3
littleh  = 0.6774
Rbox     = rbox/littleh
Sim_SFRD = BoxSfr / Rbox**3 *u.Mpc**-3
Sim_SFRD = Sim_SFRD.value

# The model comes in SFRD/DeltaZ, make sure your data does as well!! 
step_fit_logZ  = np.diff(np.log(MetalBins))[0]    
Sim_SFRD       = Sim_SFRD/step_fit_logZ

In [10]:
Sim_SFRD[99]

NameError: name 'Sim_SFRD' is not defined

In [11]:
Metals[99]

NameError: name 'Metals' is not defined

In [12]:
plt.plot(Sim_center_Zbin, Metals[99])
plt.xscale('log')
plt.yscale('log')

NameError: name 'Sim_center_Zbin' is not defined